<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dylan/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XÂY DỰNG MÔ HÌNH

## Import các thư viện cần thiết

In [11]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import string
import lightgbm as lgb
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

Mounted at /gdrive


## Đọc file dữ liệu

In [5]:
train = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/train.csv')
train

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3
...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2
17303,ffddf1f,Technology has changed a lot of ways that we l...,4
17304,fff016d,If you don't like sitting around all day than ...,2
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1


In [6]:
test = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/test.csv')
test_essay_id = test['essay_id']
test

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


## Feature engineering

Tổng hợp dữ liệu để xử lý trên các thuộc tính.



In [8]:
ntrain = train.shape[0]
ntest = test.shape[0]

df = pd.concat((train, test)).reset_index(drop=True)

### Tiền xử lý dữ liệu ký tự

-  **Chuyển chữ viết hoa thành chữ thường**

In [9]:
df['full_text'] = df['full_text'].apply(lambda x: x.lower())

- **Xóa các tag tên, hashtag**


In [12]:
df['full_text'] = df['full_text'].apply(lambda x: re.sub(r'@\w+\s*', '', x))
df['full_text'] = df['full_text'].apply(lambda x: x.replace("#", ""))

- **Xóa link URL**

In [13]:
df['full_text'] = df['full_text'].apply(lambda x: re.sub(r'http\S+|www\S+', '', x))

- **Xóa dấu câu**

In [15]:
df['full_text'] = df['full_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

- **Xóa chữ số**

In [16]:
df['full_text'] = df['full_text'].apply(lambda x: re.sub(r'\d+', '', x))

- **Xóa HTML Tags**

In [17]:
df['full_text'] = df['full_text'].apply(lambda x : re.compile(r'<.*?>').sub(r'', x))

### Chuẩn bị dữ liệu

In [19]:
train = df[:ntrain][:]
test = df[ntrain:][:]

### Thuật toán TF-IDF (Term Frequency, Inverse Document Frequency)

"TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:"
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng. Cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất. Cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

**Reference:** https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77"

In [20]:
vectorizer = TfidfVectorizer(min_df=.05)
train_tfid = vectorizer.fit_transform(train['full_text'])
test_tfid = vectorizer.transform(test['full_text'])